In [1]:
import oracledb

import pandas as pd

import datetime

In [2]:
df_runup = pd.read_excel('runup_ora_ift.xlsx')

In [3]:
df_runup

,STAND_ID,SERVICE_SHORT_NAME,STAND_TYPE,IR_ID,IR_NAME,IR_NAME_SHORT,DB_NAME,SERVER_ID,SERVER_NAME,DNS,...,FQDN,IP_A,S_CPU_COUNT,S_CPU_CORE,Textbox38,S_RAM,S_HDD,S_CREATED_T,S_UPDATED_T,DI
0,CI01995888,Платформа,MAJOR-CHECK,CI02145046,ufearuko,IFT.CORE.B1.ARCH,ufearuko,CI02145045,tkli-efs2292,vm.mos.cloud.sbrf.ru,...,tkli-efs2292.vm.mos.cloud.sbrf.ru,10.53.58.67,1,10,10,20,1840,2019-08-16 11:36:46,2022-10-27 14:28:39,t
1,CI01995888,Платформа,MAJOR-CHECK,CI02681003,ufearuko,IFT.CORE.B1.ARCH2,ufearuko,CI02681001,tklid-efs000242,vm.mos.cloud.sbrf.ru,...,tklid-efs000242.vm.mos.cloud.sbrf.ru,10.56.139.19,1,16,16,32,1264,2020-08-25 08:26:16,2022-10-27 15:14:49,t
2,CI01995888,Платформа,MAJOR-CHECK,CI02598472,ufelmuko,IFT.CORE.B1.AUD,ufelmuko,CI02598470,tklid-efs000105,vm.mos.cloud.sbrf.ru,...,tklid-efs000105.vm.mos.cloud.sbrf.ru,10.53.172.217,1,4,4,32,2332,2020-06-10 13:49:35,2022-10-27 15:08:59,t
3,CI01995888,Платформа,MAJOR-CHECK,CI02608494,arophist,IFT.CORE.B1.HISTARCH,arophist,CI02608492,tklid-efs000122,vm.mos.cloud.sbrf.ru,...,tklid-efs000122.vm.mos.cloud.sbrf.ru,10.53.238.140,1,4,4,32,600,2020-06-19 11:47:40,2022-10-27 15:12:07,t
4,CI01995888,Платформа,MAJOR-CHECK,CI02647463,aroph,IFT.CORE.B1.HISTARCH2,aroph,CI02647461,tklid-efs000188,vm.mos.cloud.sbrf.ru,...,tklid-efs000188.vm.mos.cloud.sbrf.ru,10.53.133.151,1,4,4,32,600,2020-07-27 11:57:58,2022-10-27 15:14:43,t
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,CI01995888,Платформа,MAJOR-CHECK,CI03248829,ufeluko,IFT.PCP.M.LOG,ufeluko,CI03248827,tklid-efs000740,vm.mos.cloud.sbrf.ru,...,tklid-efs000740.vm.mos.cloud.sbrf.ru,10.56.172.37,1,16,16,32,2265,2021-07-23 11:56:35,2022-10-27 15:20:00,t
69,CI01995888,Платформа,MAJOR-CHECK,CI03058797,uferoute,IFT.PCP.M.MAIN,uferoute,CI03058795,tklid-efs000628,vm.mos.cloud.sbrf.ru,...,tklid-efs000628.vm.mos.cloud.sbrf.ru,10.56.72.27,1,4,4,16,830,2021-03-30 18:35:23,2022-10-27 15:17:22,t
70,CI01995888,Платформа,MAJOR-CHECK,CI03369687,ufelmuko,IFT.PCP.S.AUD,ufelmuko,CI03369685,tklid-efs000881,vm.mos.cloud.sbrf.ru,...,tklid-efs000881.vm.mos.cloud.sbrf.ru,10.55.64.32,1,8,8,32,2067,2021-09-17 13:06:03,2022-10-27 15:22:51,t
71,CI01995888,Платформа,MAJOR-CHECK,CI03369804,ufeluko,IFT.PCP.S.LOG,ufeluko,CI03369802,tklid-efs000882,vm.mos.cloud.sbrf.ru,...,tklid-efs000882.vm.mos.cloud.sbrf.ru,10.55.58.5,1,16,16,32,43,2021-09-17 14:01:26,2022-10-27 15:22:52,t


In [4]:
user_ora = "sys"
password_ora = "p"
host_db = df_runup['FQDN']
db_name =  df_runup['IR_NAME']
port_db = '1521'

In [5]:
query_g1 = "grant select on v_$session to dbpdba"
query_g2 = "grant select on v_$process to dbpdba"
query_g3 = "grant select on gv_$session to dbpdba"
query_g4 = "grant select on gv_$process to dbpdba"
query_a1 = "CREATE AUDIT POLICY DBPDBA_KILLSES_POLICY ACTIONS execute on dbpdba.kill_session"
query_a2 = "AUDIT POLICY DBPDBA_KILLSES_POLICY"

In [6]:
query_big1 ="""
CREATE OR REPLACE PROCEDURE dbpdba.kill_session (in_sid      IN NUMBER,
                                                 in_serial   IN NUMBER)
AS
    marked_for_kill   EXCEPTION;            
    pragma exception_init (marked_for_kill, -31); 
BEGIN
    FOR i IN (SELECT B.spid,
                     A.sid,
                     A.serial#,
                     A.username,
                     A.schemaname,
                     A.osuser,
                     A.machine,
                     A.program,
                     A.module,
                     A.action,
                     A.logon_time,
                     A.status,
                     A.event,
                     A.service_name
                FROM v$session A, v$process B
               WHERE     A.paddr = B.addr
                     AND A.sid = in_sid
                     AND A.serial# = in_serial
                     AND A.username NOT IN ('SYS',
                                            'SYSTEM',
                                            'SYSMAN',
                                            'DBSNMP',
                                            'MQ_AGENT',
                                            'GGDIRECT'))
    LOOP
        BEGIN
            EXECUTE IMMEDIATE   'alter system kill session '''
                             || i.sid
                             || ','
                             || i.serial#
                             || ''' immediate';
        EXCEPTION
            WHEN marked_for_kill
            THEN
                NULL;
            WHEN OTHERS
            THEN
                RAISE;
        END;
    END LOOP;
END;
"""

In [7]:
query_big2 ="""
declare
    vsql      varchar2 (255);
    v_check   number;

    cursor usr_chk is
        select count (1)
          from dba_users
         where username in ('DBP2LT', 'DBP2IFT');

    cursor usr_cur is
        select username
          from dba_users
         where username in ('DBP2LT', 'DBP2IFT');
begin
open usr_chk;
    fetch usr_chk into v_check;

    if v_check > 0
    then
        begin
            for i in usr_cur
            loop
                vsql := 'GRANT execute on dbpdba.kill_session to ' || i.username;
                execute immediate vsql;
                dbms_output.put_line (vsql||': - done');
            end loop;
        end;
    elsif v_check = 0
    then
        dbms_output.put_line ('No users found');
    end if;
end;
"""

In [8]:
query_dict = {'query_g1':query_g1, 'query_g2':query_g2, 'query_g3':query_g3, 'query_g4':query_g4,
              'query_big1':query_big1, 'query_a1':query_a1, 'query_a2':query_a2, 'query_big1':query_big2}

In [9]:
for i in query_dict:
    print(i)

query_g1
query_g2
query_g3
query_g4
query_big1
query_a1
query_a2


In [10]:
def execute_query(cursor_f, query_dict_f, host_db_f, db_name_f):
    
    for i in query_dict_f:
        try:
            cursor.execute(query_dict_f[i])
        except:
            str_error_query = host_db_f + '||' + db_name_f + '||упал скрипт:' + str(i)
            with open('Косячные_скрипты.txt', 'a') as f:
                    f.write(str(str_error_query) +' \n')
            print(str_error_query)

In [11]:
for i in range(len(df_runup)):
    try:
        connection = oracledb.connect(user=user_ora, password=password_ora, host=host_db[i],
                                              port=port_db,
                                              service_name=db_name[i], mode=oracledb.AUTH_MODE_SYSDBA)

        cursor = connection.cursor()
        execute_query(cursor_f=cursor, query_dict_f=query_dict, host_db_f=host_db[i], db_name_f=db_name[i])
        cursor.close()
        connection.close()
    except:
        str_error_connect = str(str(i) + ' Коннекта нет: ' + host_db[i] + '||' + db_name[i]) 
        with open('Косячные_подключения.txt', 'a') as f:
            f.write(str(str_error_connect) +' \n')
        print(str_error_connect)

tkli-efs2292.vm.mos.cloud.sbrf.ru||ufearuko||упал скрипт:query_a1
tklid-efs000242.vm.mos.cloud.sbrf.ru||ufearuko||упал скрипт:query_a1
tklid-efs000105.vm.mos.cloud.sbrf.ru||ufelmuko||упал скрипт:query_a1
tklid-efs000122.vm.mos.cloud.sbrf.ru||arophist||упал скрипт:query_a1
tklid-efs000188.vm.mos.cloud.sbrf.ru||aroph||упал скрипт:query_a1
tklid-efs000119.vm.mos.cloud.sbrf.ru||unophist||упал скрипт:query_a1
tvldd-efs000447.delta.sbrf.ru||ufeluko||упал скрипт:query_a1
tklid-efs000609.vm.mos.cloud.sbrf.ru||ufeuko||упал скрипт:query_a1
tkli-efs2287.vm.mos.cloud.sbrf.ru||ufeuko||упал скрипт:query_g1
tkli-efs2287.vm.mos.cloud.sbrf.ru||ufeuko||упал скрипт:query_g2
tkli-efs2287.vm.mos.cloud.sbrf.ru||ufeuko||упал скрипт:query_g3
tkli-efs2287.vm.mos.cloud.sbrf.ru||ufeuko||упал скрипт:query_g4
tkli-efs2287.vm.mos.cloud.sbrf.ru||ufeuko||упал скрипт:query_big1
tkli-efs2287.vm.mos.cloud.sbrf.ru||ufeuko||упал скрипт:query_a1
tkli-efs2287.vm.mos.cloud.sbrf.ru||ufeuko||упал скрипт:query_a2
tklid-efs00011

In [ ]:
connection = oracledb.connect(user=user_ora, password=password_ora, host='medusa34.delta.sbrf.ru',
                                              port=port_db,
                                              service_name='fllogbus', mode=oracledb.AUTH_MODE_SYSDBA)
cursor = connection.cursor()

execute_query(cursor_f=cursor, query_dict_f=query_dict, host_db_f='medusa34.delta.sbrf.ru', db_name_f='fllogbus')